In [ ]:
from hashlib import md5
# from sklearn import tree
import pandas as pd
import numpy as np
import openpyxl
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LogisticRegression
from pathlib import Path
from datetime import datetime

In [ ]:
def normalize(X):
    for i in range(X.shape[1]):
        X[:,i] = (X[:,i] - X[:,i].mean())/X[:,i].std()
    return X

In [ ]:
k = 10
p = 8
def get_data():
    """
    Returns the data from the xlsx file
    """
    file_name = 'yeast.csv' 
    df = pd.read_csv(file_name, index_col=0, header=None)
    # print(df.head()) # print the first 5 rows
    return df

md = get_data()
md.dropna(inplace = True)
print(md.shape)
md.replace('?', 0, inplace = True)

print(md)

In [ ]:
X = md.iloc[:, 0:8].values.reshape(-1, 8)


print(X)
print(X.shape)
# add 1 to the first column
# X = np.insert(X, 0, 1, axis=1)
# np.random.shuffle(X)

print('X:', X)
print(X.shape)
from sklearn import preprocessing

# print(preprocessing.scale(X))
X = normalize(X)
print(X)
# T = preprocessing.scale(X)

M = X.shape[0]
Y = md.iloc[:, 8:9].values.reshape(-1, 1)


In [ ]:
def p(i):
    return 1 / (1 + np.exp(-i))

In [ ]:
outputs = ['CYT', 'NUC', 'MIT', 'ME3', 'ME2', 'ME1', 'EXC', 'VAC', 'POX', 'ERL']

y = np.zeros((len(Y), 10))
print(y.shape)

for i in range(len(Y)):
    for j in range(10):
        if outputs[j] == Y[i]:
            y[i][j] = 1
        else:
            y[i][j] = 0

print('results', Y[0] == outputs[2])
print(y.shape)
# for i in range(10):
#     print(outputs[i], ': ', y[i])

In [7]:
eta = 1
beta = np.zeros((10, 8))

for i in range(10):
    beta[i] = np.random.rand(1, 8)

print(beta)
gradient = np.zeros((10, 8))
trainNo = 1000
loopCount = 1000

n = 0
while (True):
    for i in range(trainNo):   
        tmp = p(X[i] @ beta.T)
        tmp = tmp - y[i]
        for t in range(k):
            gradient[t] = gradient[t] + tmp[t] * X[i]
    gradient = gradient / trainNo
    
    
    beta = beta - eta * gradient
    print('g: ', gradient, 'norm', np.linalg.norm(gradient))

    n = n + 1
    if n == trainNo or np.linalg.norm(gradient) < 0.0097:
        break
    eta = eta * 0.99

print('beta: ', beta)

g:  [[ 1.46860739e-03 -1.71796217e-03 -3.58698789e-04  4.77333814e-04
   4.46787795e-05  1.55632550e-04  5.92199604e-06  8.03310774e-04]
 [-2.71824346e-03  3.14559180e-03  7.59117313e-04  4.21818769e-04
   1.46765213e-04  3.40667769e-04  1.56290754e-03 -4.99065885e-03]
 [ 2.81122178e-04  7.89721861e-04  6.84321913e-04 -1.05189271e-03
  -6.97108855e-05  4.24009145e-04 -4.14617959e-04  1.63729400e-03]
 [ 2.15509249e-03 -3.09425130e-04  3.90648867e-03  2.60775064e-04
  -1.21698183e-04  1.96208601e-05  9.22099001e-04  8.51414528e-04]
 [-6.99259624e-04  1.25349221e-03  9.39160326e-04 -1.48398526e-04
  -8.48708938e-05  5.26367876e-05  2.38693824e-04  3.93525803e-04]
 [ 4.19665801e-06  6.13419625e-04  8.93568867e-04 -1.17109166e-04
  -1.45300766e-04  2.67106465e-04  2.44819012e-04  8.42238330e-04]
 [-1.25210957e-06  1.30852466e-04  4.54157630e-04  6.49721150e-05
   6.49455133e-05  3.10254389e-04  2.67383133e-04  2.93732624e-04]
 [-7.35112099e-04  1.49326582e-03  9.47283259e-04 -2.04536971e-04

In [8]:
# eta = 0.001
# trainNo = 1200
# beta = np.zeros((10, 9))
# for t in range(10):
#     for i in range(trainNo):
#         # take one sample randomly
#         # i = np.random.randint(0, trainNo)
#         print(i)
        
#         # calculate the gradient
#         for k in range(10):
#             for j in range(9):
#                 gradient[k][j] = (p(np.dot(X[i], beta[k])) - y[i][k]) * X[i][j]

#         print('grad', gradient)

#         # update beta
#     for k in range (10):
#         beta[k] = beta[k] - eta * gradient[k] / trainNo

In [9]:
# one or not classification
x_pred = np.zeros(trainNo)
estimates = np.zeros(10)

for i in range(M - trainNo):
    for t in range(10):
        estimates[t] = p(np.dot(X[trainNo + i], beta[t]))
    x_pred[i] = np.argmax(estimates)
    # print(estimates)
    # print(x_pred[i])
print(x_pred)

correct = 0
for i in range(M - trainNo):
    if np.where(y[trainNo + i] == 1) == x_pred[i]:
        correct = correct + 1
print(correct / (M - trainNo), (M - trainNo))


[2. 2. 2. 1. 2. 1. 0. 0. 0. 0. 2. 0. 0. 0. 2. 2. 0. 6. 2. 2. 1. 0. 0. 1.
 2. 2. 1. 1. 1. 0. 1. 1. 2. 3. 1. 1. 6. 0. 1. 2. 0. 3. 2. 0. 2. 2. 2. 2.
 0. 1. 4. 3. 0. 0. 0. 0. 0. 3. 9. 2. 3. 3. 2. 1. 3. 0. 3. 3. 3. 0. 3. 3.
 0. 1. 1. 0. 6. 9. 3. 1. 3. 1. 0. 2. 6. 4. 3. 2. 3. 3. 2. 1. 1. 0. 2. 2.
 1. 1. 1. 0. 1. 3. 3. 0. 1. 1. 1. 2. 3. 0. 1. 3. 1. 3. 1. 3. 3. 0. 2. 3.
 3. 2. 1. 3. 3. 1. 1. 0. 0. 0. 3. 0. 2. 1. 3. 1. 0. 0. 0. 1. 3. 1. 2. 2.
 3. 8. 1. 3. 3. 1. 0. 3. 3. 0. 1. 1. 1. 0. 0. 3. 0. 0. 0. 1. 2. 1. 3. 1.
 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 0. 2. 1. 1. 1. 3. 3. 2. 3. 3. 6. 1. 3. 3.
 1. 3. 1. 4. 1. 1. 3. 2. 1. 1. 3. 1. 3. 3. 3. 6. 6. 6. 0. 0. 0. 1. 3. 0.
 1. 3. 2. 2. 3. 0. 1. 1. 0. 3. 3. 6. 3. 3. 1. 3. 0. 0. 4. 3. 0. 2. 2. 2.
 1. 0. 0. 0. 1. 1. 2. 1. 1. 1. 1. 1. 1. 0. 2. 0. 3. 0. 3. 1. 1. 1. 1. 0.
 0. 4. 0. 4. 6. 0. 0. 0. 1. 0. 1. 1. 0. 2. 2. 0. 0. 0. 0. 0. 0. 1. 1. 1.
 1. 0. 1. 0. 1. 0. 2. 0. 3. 1. 2. 4. 0. 0. 0. 0. 2. 1. 1. 0. 1. 0. 0. 0.
 3. 0. 0. 1. 0. 1. 1. 1. 3. 1. 1. 2. 0. 0. 0. 2. 1.